In [ ]:
import math
import numpy as np
from shapely.geometry import Point
from shapely.geometry.polygon import LinearRing

def reward_function(params):

    MAX_SPEED = 3.5  # Maximum speed (used for straight portions of the track)
    MIN_SPEED = 1.0  # Minimum speed (used for turns)
    ABS_STEERING_THRESHOLD = 15
    track_curve_threshold = math.radians(15)

    track_width = params['track_width']
    distance_from_center = params['distance_from_center']
    position = Point(params['x'], params['y'])
    track = LinearRing(params['waypoints'])
    steering = abs(params['steering_angle'])
    speed = params['speed']
    on_track = params['all_wheels_on_track']
    on_left = params['is_left_of_center']
    waypoints = params['waypoints']
    closest_waypoints = params['closest_waypoints']
    
    point = position.distance(track)

    reward = 1e-3

    if steering > ABS_STEERING_THRESHOLD:
        reward *= 0.8

    if not on_track:
        reward = 1e-3
    else:
        reward += 0.1
    
    if on_left:
        reward *= 0.8
    else:
        reward += 0.1
        
    if point < 1:
        reward += 1.2
    elif point < 3:
        reward += 0.9
    elif point < 6:
        reward += 0.6
    elif point < 9:
        reward += 0.3   
    else:
        reward *= 0.3

    marker_rewards = [('marker', float), ('marker_reward', float)]

    # Create the structured array
    markers = np.array([
        (0.05 * track_width, 2.0),
        (0.10 * track_width, 1.6),
        (0.15 * track_width, 1.4),
        (0.20 * track_width, 1.2),
        (0.25 * track_width, 0.8),
        (0.30 * track_width, 0.6),
        (0.35 * track_width, 0.4),
        (0.40 * track_width, 0.2),
        (0.50 * track_width, 0.1)
    ], dtype=marker_rewards)

    for marker in markers:
        marker_value, marker_reward = marker
        if distance_from_center <= marker_value:
            reward *= marker_reward
            break

    current_waypoint = waypoints[closest_waypoints[0]] # index 0 for closest
    next_waypoint = waypoints[closest_waypoints[1]] # index 1 for next closest
    next_next_waypoint_index = (closest_waypoints[1] + 1) % len(waypoints)
    next_next_waypoint = waypoints[next_next_waypoint_index]

    direction_vector = [next_waypoint[0] - current_waypoint[0], next_waypoint[1] - current_waypoint[1]]

    next_direction_vector = [next_next_waypoint[0] - next_waypoint[0], next_next_waypoint[1] - next_waypoint[1]]

    # If you have problems to validate here do 
    # math.atan2((next_direction_vector[1], next_direction_vector[0]) - (direction_vector[1], direction_vector[0]))
    track_angle = math.atan2(next_direction_vector[1], next_direction_vector[0]) - math.atan2(direction_vector[1], direction_vector[0])

    track_angle = abs(track_angle)

    if track_angle < track_curve_threshold:
        # Large reward for max speed on straight tracks
        if speed >= MAX_SPEED:
            reward *= 1.15
    else:
        # Large reward for slower speed on turns/curves
        if speed <= MIN_SPEED:
            reward *= 1.10

    return float(reward)

